In [ ]:
# wanddb
# Mixup + Cutmix
# Pseudo Label, TF Serving, gpustat
# SageMaker BiT, GladCam, BYOS, Debugger

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import configparser
import os
import shutil
import sys
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_hub as hub
from sklearn.model_selection import KFold
from albumentations import (
    Compose,
    HorizontalFlip,
    HueSaturationValue,
    ImageCompression,
    RandomBrightnessContrast,
    Rotate,
)

sys.path.append("..")
from utils.common import get_cpu_count
from utils.data_utils import (
    _parse_function,
    dump_tfrecord,
    load_tfrecord_dataset,
)

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
# warnings.filterwarnings("ignore")
np.random.seed(42)

In [ ]:
def view_image(dataset, n_samples, label_names=None):
    images, labels = next(iter(dataset))
    images = images.numpy()
    labels = labels.numpy()

    n_cols = 5
    n_rows = n_samples // n_cols if n_samples % n_cols == 0 else n_samples // n_cols + 1
    fig = plt.figure(figsize=(n_cols * 4, n_rows * 3))
    for i in range(n_samples):
        ax = fig.add_subplot(n_rows, n_cols, i + 1, xticks=[], yticks=[])
        ax.imshow(images[i])
        label = (
            labels[i].decode("utf-8")
            if label_names is None
            else label_names["c" + str(np.argmax(labels[i]))]
        )
        ax.set_title(f"Label: {label}")
        ax.axis("off")

In [ ]:
config = configparser.ConfigParser()
_ = config.read(os.path.join("..", "conf", "config.ini"))

raw_data_path = config["project"]["raw_data_path"]
validate_by_driver = eval(config["project"]["validate_by_driver"])
n_tfrec_chunks = eval(config["project"]["n_tfrec_chunks"])

model_url = config["model"]["model_url"]
trainable = eval(config["model"]["trainable"])
img_size = eval(config["model"]["img_size"])
n_epochs = eval(config["model"]["n_epochs"])
batch_size = eval(config["model"]["batch_size"])
initial_learning_rate = eval(config["model"]["initial_learning_rate"])
first_decay_steps = eval(config["model"]["first_decay_steps"])
use_adamw = eval(config["model"]["use_adamw"])
use_swa = eval(config["model"]["use_swa"])
label_smoothing = eval(config["model"]["label_smoothing"])
n_train_splits = eval(config["model"]["n_train_splits"])
n_test_splits = eval(config["model"]["n_test_splits"])

label_names = {
    "c0": "safe driving",
    "c1": "texting - right",
    "c2": "talking on the phone - right",
    "c3": "texting - left",
    "c4": "talking on the phone - left",
    "c5": "operating the radio",
    "c6": "drinking",
    "c7": "reaching behind",
    "c8": "hair and makeup",
    "c9": "talking to passenger",
}

labels = list(label_names.keys())
num_classes = len(label_names)

## Data Preprocessing
### *TFRecord* Files Creation

In [ ]:
train_raw_data_path = os.path.join(raw_data_path, "imgs", "train")

train_examples = []
for label in os.listdir(train_raw_data_path):
    img_paths = tf.io.gfile.glob(os.path.join(train_raw_data_path, label, "*.jpg"))
    for img_path in img_paths:
        file_name = os.path.join(label, os.path.basename(img_path))
        train_examples.append((img_path, label, file_name))

n_train_examples = len(train_examples)

train_proc_data_path = os.path.join(raw_data_path, "tfrec", "train")
if os.path.exists(train_proc_data_path):
    shutil.rmtree(train_proc_data_path)

if validate_by_driver:
    driver_imgs_list = pd.read_csv(os.path.join(raw_data_path, "driver_imgs_list.csv"))
    driver_imgs_list["img_path"] = (
        driver_imgs_list["classname"] + os.path.sep + driver_imgs_list["img"]
    )
    img_paths_by_driver = (
        driver_imgs_list.groupby("subject")["img_path"].apply(list).to_dict()
    )

    for key, value in img_paths_by_driver.items():
        train_examples_by_driver = [
            example for example in train_examples if example[2] in value
        ]
        np.random.shuffle(train_examples_by_driver)
        dump_tfrecord(
            train_examples_by_driver,
            os.path.join(train_proc_data_path, f"{key}.tfrec"),
            num_classes=num_classes,
        )

else:
    np.random.shuffle(train_examples)
    for i in range(n_tfrec_chunks):
        if i == 0:
            start = 0
        else:
            start = end
        if i == n_tfrec_chunks - 1:
            end = n_train_examples
        else:
            end = (i + 1) * (n_train_examples // n_tfrec_chunks)

        dump_tfrecord(
            train_examples[start:end],
            os.path.join(train_proc_data_path, f"{str(i).zfill(2)}.tfrec"),
            num_classes=num_classes, 
            is_prediction=False,
        )

In [ ]:
test_raw_data_path = os.path.join(raw_data_path, "imgs", "test")

test_examples = []
img_paths = np.sort(
    tf.io.gfile.glob(os.path.join(test_raw_data_path, "*.jpg"))
).tolist()
for img_path in img_paths:
    file_name = os.path.basename(img_path)
    test_examples.append((img_path, file_name))

n_test_examples = len(test_examples)

test_proc_data_path = os.path.join(raw_data_path, "tfrec", "test")
if os.path.exists(test_proc_data_path):
    shutil.rmtree(test_proc_data_path)

for i in range(n_tfrec_chunks):
    if i == 0:
        start = 0
    else:
        start = end
    if i == n_tfrec_chunks - 1:
        end = n_test_examples
    else:
        end = (i + 1) * (n_test_examples // n_tfrec_chunks)

    dump_tfrecord(
        test_examples[start:end],
        os.path.join(test_proc_data_path, f"{str(i).zfill(2)}.tfrec"),
        is_prediction=True,
    )

### The Input Data Pipeline Configuration with *Dataset* API

In [ ]:
train_tfrec_paths = tf.io.gfile.glob(
    os.path.join(raw_data_path, "tfrec", "train", "*.tfrec")
)
test_tfrec_paths = tf.io.gfile.glob(
    os.path.join(raw_data_path, "tfrec", "test", "*.tfrec")
)

train_transforms = Compose(
    [
        Rotate(limit=20),
        RandomBrightnessContrast(brightness_limit=0.1),
        ImageCompression(quality_lower=85, quality_upper=100, p=0.5),
        HueSaturationValue(
            hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5
        ),
        RandomBrightnessContrast(contrast_limit=0.2, p=0.5),
        HorizontalFlip(),
    ]
)
valid_transforms = train_transforms
test_transforms = train_transforms

n_samples = 20
sampled_dataset = load_tfrecord_dataset(
    [train_tfrec_paths[0]],
    (480, 640),
    train_transforms,
    n_samples,
    num_classes=num_classes,
)

view_image(sampled_dataset, n_samples, label_names)

Model

In [ ]:
model = tf.keras.Sequential(
    [
        hub.KerasLayer(
            model_url, trainable=trainable, input_shape=(img_size, img_size, 3)
        ),
        tf.keras.layers.Dense(num_classes, activation="softmax"),
    ]
)
print(model.summary())

# For tensorflow 2.5 or later, use tf.keras.optimizers.schedules.CosineDecayRestarts.
lr_decayed_fn = tf.keras.experimental.CosineDecayRestarts(
    initial_learning_rate, first_decay_steps
)
optimizer = (
    tfa.optimizers.AdamW(lr_decayed_fn)
    if use_adamw
    else tfa.optimizers.RectifiedAdam(lr_decayed_fn)
)
if use_swa:
    optimizer = tfa.optimizers.SWA(optimizer, start_averaging=0, average_period=10)

model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=label_smoothing),
    metrics=["acc"],
)

In [ ]:
train_steps_per_epoch = round(
    n_train_examples * (n_train_splits - 1) / n_train_splits / batch_size
)
valid_steps_per_epoch = round(n_train_examples / n_train_splits / batch_size)

kf = KFold(n_splits=n_train_splits, shuffle=True, random_state=42)

model_path = os.path.join("..", "models")
os.makedirs(model_path, exist_ok=True)

test_dataset = load_tfrecord_dataset(
    test_tfrec_paths,
    img_size,
    test_transforms,
    1,
    shuffle=False,
    num_classes=num_classes,
    is_prediction=True,
)
workers = get_cpu_count(0.9)

for i, (train_index, valid_index) in enumerate(kf.split(range(len(train_tfrec_paths)))):
    train_dataset = load_tfrecord_dataset(
        np.array(train_tfrec_paths)[train_index],
        img_size,
        train_transforms,
        batch_size,
        num_classes=num_classes,
    )
    valid_dataset = load_tfrecord_dataset(
        np.array(train_tfrec_paths)[valid_index],
        img_size,
        valid_transforms,
        batch_size,
        num_classes=num_classes,
    )

    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=3, restore_best_weights=True
        ),
        tf.keras.callbacks.ModelCheckpoint(
            os.path.join(model_path, f"{str(i).zfill(2)}.h5"), monitor="val_loss"
        ),
    ]

    model.fit(
        train_dataset,
        epochs=n_epochs,
        steps_per_epoch=train_steps_per_epoch,
        validation_data=valid_dataset,
        validation_steps=valid_steps_per_epoch,
        callbacks=callbacks,
    )

    for j in range(n_test_splits):
        prediction = model.predict(
            test_dataset,
            steps=n_test_examples,
            workers=workers,
            use_multiprocessing=True,
        )

        if j == 0:
            result = pd.DataFrame(prediction, columns=labels)
        else:
            result += pd.DataFrame(prediction, columns=labels)